<div align="center">
    <h1>Variational Inference and Generative Modelling Tutorial</h1>
    <h3> Weizmann AI Hub for Scientific Discovery </h3>
    <h4>Nathan LEVY</h4>
    nathan.levy@weizmann.ac.il
    <p>with inputs from N.Yosef and M.Kim</p>
    <p>Spring semester 2024</p>
</div>




In this tutorial, we are going to delve into a class of generative models called Variational Auto-Encoders. We start by stating key statistical concepts before deriving the relevant mathematical formulation. Then we build and train a VAE. \
Finally, we are going to apply the models we built to single-cell RNA-seq data, and show how these models can be used for representation learning.

💡 This tutorial does not assume prior knowledge in variational inference or biology, but does rely on basic deep learning skills, such as building and training a feedforward neural network. AI hub students should use the `scvi-env` environment. 

# 1. Introduction

To begin with, let us review a few useful ML concepts. 

### 1.1  Discriminative versus Generative learning


**Discriminative Learning:** \
Discriminative learning is a machine learning approach that focuses on learning the boundary or decision boundary that separates different classes or categories within the data. The primary goal of discriminative models is to find a function that directly maps the input data to the corresponding class labels. These models are designed to distinguish between different classes based on their distinct features or patterns.

A basic example of discriminative model is logistic regression, it is often used in classification tasks where the goal is to assign input data points to specific predefined categories or classes.



**Generative Learning:**\
Generative learning, on the other hand, is a machine learning approach that aims to model the underlying probability distribution of the entire dataset. Instead of focusing solely on learning the decision boundary, generative models attempt to capture the inherent structure and patterns within the data. This allows them to generate new samples that resemble the original data distribution.

Generative models can be used for various tasks, including data synthesis, image generation, and anomaly detection. Some popular generative models include Gaussian Mixture Models (GMMs), Variational Autoencoders (VAEs), and Generative Adversarial Networks (GANs).

In summary, discriminative learning only models the outputs given the inputs, while generative learning models the joint probability of inputs and outputs. 


The focus of this tutorial is on generative learning. To fit the generative models, we adopt the Bayesian inference paradigm.

### 1.2 Bayesian modelling and Variational Inference


Suppose we have a probabilistic model with some parameters we want to estimate. The Bayesian paradigm involves the following key concepts:

1. **Prior Probability:** Before observing any new data, we have an initial belief about our model parameters $\theta$. This belief is expressed as the prior probability $p(\theta)$.

2. **Likelihood:** Given a dataset $D$, the likelihood function $L$ describes how likely the data is to occur given the underlying parameters: $L=p(D \mid \theta)$

3. **Posterior Probability:** After observing new data, the Bayesian approach updates the initial beliefs to incorporate information from this data. This process is called *Bayesian inference*. The updated parameters are represented as the posterior probability $p(\theta \mid D)$, which combines the prior probability and the likelihood.

4. **Evidence:** The term $p(D)$ represents the likelihood of observing the data across all possible values of the parameters or hypotheses. It acts as a normalizing constant to ensure that the posterior probability integrates to 1. The main goal of any Bayesian count_model is to optimize the evidence for the data, which is also known as the marginal likelihood. It is often more convenient to work with the log marginal likelihood.


The posterior probability is computed via the *Bayes rule*:

$$ p(\theta \mid D) = \frac{p(D|θ)p(θ)}{\int p(D|θ)p(θ)dθ} = \frac{p(D|θ)p(θ)}{p(D)}$$


However, in most models, the posterior distribution is intractable because of the integral defining the evidence, which runs over the space of all $\theta$. Therefore, the posterior must be estimated: this is called *approximate Bayesian inference*. 

There are a few popular approximate Bayesian inference methods, among which Markov Chain Monte Carlo (the idea is to construct a Markov chain that has the posterior distribution as its equilibrium distribution) and Variational Inference, which tries to minimize the distance between an approximate, tractable posterior (called the variational distribution) and the true one. We are going to focus on the latter. 


### 1.3  Latent Variable Models

Given observed data $D$, Latent Variable Models assume that the data generating process is driven by unobserved *latent variables* $Z$, of smaller dimension. The latent variables should be a summary of the data, driving its variability. They can represent a useful representation of the data. 


Mixture Models are a classsic example of LVMs. They assume that the dataset $D$ is generated by sampling i.i.d. from $K$ distincts distributions called mixture components. We can describe the data generating process as follows: 

1. For each datapoint, we first sample the mixture assignment from a categorical random variable, out of $K$ possible components. 
2. Then, we sample from the assigned distribution. 

In this count_model, we do observe the datapoints, which were drawn conditionnally on the mixture assignment, but we do not observe the assignment itself: it is the latent variable. 

Given the observed data, we want to infer the latent variables, using the posterior distribution $p(Z \mid D)$. For this purpose, we'll have to rely on approximate Bayesian inference, more precisely Variational Inference. Thus we introduce the approximate posterior $q(Z \mid D)$, which comes from a family of tractable distributions. As we said, the goal is to minimize the distance between the true and approximate posterior. The common metric to use is the [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence). It is defined as: 
$$\mathbb{KL}(q \mid p) = \int q(x) \log \frac{q(x)}{p(x)} dx$$

The Kullback-Leibler divergence is a measure of how one probability distribution diverges from a second, expected probability distribution. In other words, it is a measure of how much information is lost when the second distribution is used to approximate the first one.

<div class="alert alert-block alert-info">
<b>Question 1</b>
Is the KL divergence symmetric? What is its value when the two distributions are equal? What is its minimum value?
</div>

#### Evidence Lower Bound (ELBO)

We want to fit out latent variable count_model to the data, but the log marginal likelihood is intractable. We also want to evaluate the posterior distribution $p(Z \mid D)$, but it is also intractable, as its denominator is the evidence. Instead, we are going to maximize a lower bound on the log marginal likelihood, called the Evidence Lower Bound (ELBO).

Recall from Bayes rule that the log marginal likelihood can be written as:
$$\log p(D) = \log\frac{p(D \mid Z)p(Z)}{p(Z \mid D)} $$

We take the expectation with respect to the approximate posterior $q(Z \mid D)$ - we write this operation $\mathbb{E}_{Z \sim q}[\cdot]$, which gives the same result as integrating over $Z$ with respect to $q(Z \mid D)$:

$$\log \left[ p(D) \right] = \mathbb{E}_{Z \sim q} \left[ \log\frac{p(D \mid Z)p(Z)}{p(Z \mid D)} \right]$$

Multiplying and dividing by $q(Z \mid D)$, we get:

$$\log \left[ p(D) \right] = \mathbb{E}_{Z \sim q} \left[ \log\frac{p(D \mid Z)p(Z)}{p(Z \mid D)} \frac {q(Z|D)}{q(Z|D)} \right]$$

Thus we can decompose the log marginal likelihood as follows:

$$\log p(D) = \mathbb{E}_{Z \sim q} \log \frac{q(Z|D)}{p(Z|D)} + \mathbb{E}_{Z \sim q} \log \frac{p(Z)}{q(Z|D)} + \mathbb{E}_{Z \sim q} \log p(D | Z)$$

This can be re-expressed using the KL divergence as follows:

$$\log p(D) = \mathbb{KL}(q(Z | D) \| p(Z | D)) - \mathbb{KL}(q(Z | D) \| p(Z)) + \mathbb{E}_{Z \sim q} \log p(D|Z)$$

Because the KL divergence is always positive, we get the following inequality:
$$\log p(D) \geq - \mathbb{KL}(q(Z | D) \| p(Z)) + \mathbb{E}_{Z \sim q} \log p(D|Z)$$

The right-hand side of the inequality is called the Evidence Lower Bound [(ELBO)](https://en.wikipedia.org/wiki/Evidence_lower_bound). It is a lower bound on the log marginal likelihood. By maximizing the ELBO, we avoid the intractability of the log marginal likelihood. 

$$ELBO =  - \mathbb{KL}(q(Z | D) \| p(Z)) + \mathbb{E}_{Z \sim q} \log p(D|Z)$$

The ELBO includes two sets of parameters: the parameters  $\phi$ of the approximate posterior $q$ and the parameters  $\theta$ of likelihood $p$.

### 1.4 Deep Generative Models (DGMs)

Deep generative models are generative models that assume that both distributions $q$ and $p$ are parameterized by neural networks.
Variational Autoencoders (VAEs) are a class of DGMs with two key components: an encoder and a decoder. The encoder maps the input data to a latent representation, while the decoder maps the latent representation back to the input space. The encoder $f_\phi$ and decoder $f_\theta$ are trained jointly to maximize the ELBO.

A typical choice for the latent prior $p(Z)$ is the standard isotropic Gaussian $\mathcal{N}(0,I)$, while the appoximate posterior can be chosen as diagonal Gaussian
$$q(Z \mid D) = \mathcal{N}(\mu_\phi(D), \sigma_\phi(D))$$

such that:

$$\mu_\phi(D), \sigma_\phi(D) = f_\phi(D)$$

Where $f_\phi$ is a neural network with parameters $\phi$ and $\sigma_\phi(D)$ is a diagonal covariance matrix with zero off-diagonal elements.

With these choices, the KL divergence term in the ELBO can be computed in closed form.
The form of decoder distribution changes with the type of data we want to model. For example, if we want to model **binary data**, we can use a Bernoulli distribution for the decoder: 
$$p(D \mid Z) = \mathcal{B}(\mu_\theta(Z))$$
With $\mu_\theta(Z) = f_\theta(Z)$.

If we want to model **continuous data**, we can use a (diagonal) Gaussian distribution for the decoder, such that:
$$p(D \mid Z) = \mathcal{N}(\mu_\theta(Z), \sigma_\theta(Z))$$
With $\mu_\theta(Z), \sigma_\theta(Z) = f_\theta(Z)$.

Finally, for **count data**, such as gene expression counts in single-cell RNA sequencing, a Poisson distribution is often an appropriate choice for the decoder. In this case, the decoder distribution can be modeled as:

$$p(D \mid Z) = \text{Poisson}(\lambda_\theta(Z))$$

Where $\lambda_\theta(Z) = f_\theta(Z)$ represents the rate parameter of the Poisson distribution. This parameter is typically interpreted as the expected count or mean of the distribution, and it is always non-negative. The function $f_\theta(Z)$ is usually implemented as a neural network that maps the latent space Z to the positive real numbers, often using an exponential or softplus activation function in the final layer to ensure non-negativity.


Using the 3 distributions above, we can analytically compute the two ELBO terms: the KL $\mathbb{KL}(q(Z | D) \| p(Z))$ and the reconstruction term $\mathbb{E}_{Z \sim q} \log p(D|Z)$.

<div class="alert alert-block alert-info">
<b>Question 2</b>
What is the analytical expression of the KL divergence? What is the analytical expression of the reconstruction term for a Gaussian decoder? For a Bernoulli decoder?

💡 Hints: minimizing the negative Bernoulli log-likelihood is equivalent to minimizing the binary cross-entropy loss. Minimizing the negative Gaussian log-likelihood is equivalent to minimizing the mean squared error for unit variance. You may also check the VAE seminal paper https://arxiv.org/pdf/1312.6114v10.pdf. 
</div>

#### Stochastic optimization of the ELBO

We want to use stochastic gradient descent to optimize the ELBO. For this purpose, we need to compute the gradient of the ELBO with respect to the parameters $\phi$ and $\theta$. However, we cannot differentiate the ELBO w.r.t. $\phi$ as we cannot directly backpropagate
gradients through the random variable $Z$. We will need to use a change of variables called the *reparametrization trick*. Please read about the trick [here](https://sassafras13.github.io/ReparamTrick/).

# 2. Design a VAE

Now that we derived the mathematical formulation of the VAE, we are going to implement it using PyTorch. The code to complete is contained in `base_components.py`, `model.py` and `train.py`. Utility functions are provided in `utils.py`. All files are auto-imported in the notebook. It means that you can change them, save them (`Ctrl+S`) and this change will immediately take effect in the notebook.

We focus on single-cell RNA-seq data. The data consists in  a matrix of gene expression counts, where each row corresponds to a cell and each column corresponds to a gene. The goal is to learn a low-dimensional representation of the cells, which captures the main sources of variability in the data.

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import matplotlib.pyplot as plt
from rich import print

## 2.1 Build the Encoder and the Decoder

**General instructions:**

- You are free to use whatever you prefer to build the model layers, but we provide you with a helper function `make_layer`, with default ReLU and BatchNorm layers.

In [73]:
# #example usage of make_layer

# from base_components import make_layer

# input_size = 784
# hidden_sizes = [256, 128]
# output_size = 2

# layer_sizes = [input_size, *hidden_sizes, output_size]

# layers = torch.nn.Sequential(
#     *[
#         make_layer(
#             in_dim=in_size,
#             out_dim=out_size,
#         )
#         for (in_size, out_size) in zip(layer_sizes[:-1], layer_sizes[1:])
#     ]
# )

- You can either code the reparameterization trick or use the rsample method of torch.distributions.Normal. This method returns a sample from the distribution, with gradients flowing through the mean and standard deviation. You can have a look at the documentation [here](https://pytorch.org/docs/stable/distributions.html). 

<div class="alert alert-block alert-info">
<b>Question 3</b>
Complete the code in `base_components.py` to build the encoder and the decoder.
</div>

## 2.2 Build the VAE model

- The VAE model is built by combining the encoder and the decoder.


<div class="alert alert-block alert-info">
<b>Question 4</b>
Complete the code in `model.py` to build the VAE model.
</div>

# 3. Applying DGMs to sc-RNA-seq data

Setup (change the params)

In [ ]:
BATCH_SIZE = 1
LATENT_DIM = 1
N_HIDDEN = 1
MAX_KL_WEIGHT = 1
N_EPOCHS = 1
LR = 1
BATCH_KEY = "batch"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {DEVICE} device")

## 3.1 Data Handling

In [74]:
import anndata as ad

import scanpy as sc
import numpy as np

import warnings

warnings.filterwarnings("ignore")

We consider the human lung cell atlas introduced in [Luecken et al. 2021](https://www.nature.com/articles/s41592-021-01336-8).

In [ ]:
adata = sc.read(
    "lung_atlas.h5ad",
    backup_url="https://figshare.com/ndownloader/files/24539942",
)
print(adata)

In [76]:
adata.layers["counts"] = adata.X.copy()

In [ ]:
adata.layers["counts"]  # .todense()

### Basic preprocessing

We perform basic preprocessing on the sc-RNA-seq data. We filter out lowly expressed genes. In order to compute a PCA later on, we normalize and log-transform the data. 

💡 More details on these steps can be found on the single-cell integration tutorial. 

In [79]:
sc.pp.filter_genes(adata, min_counts=5)
sc.pp.filter_cells(adata, min_counts=5)
sc.pp.normalize_total(adata, target_sum=1e4)

In [80]:
sc.pp.highly_variable_genes(
    adata,
    n_top_genes=1200,
    subset=True,
    layer="counts",
    flavor="seurat_v3",
    batch_key="batch",
)

### Build the dataloader

We use [AnnDataLoader](https://docs.scvi-tools.org/en/stable/tutorials/notebooks/dev/data_tutorial.html#recording-anndata-state-with-object-registration) from scvi-tools to build the dataloader. We will select the subsets of the AnnData fthat are needed for training and inference. Of course we'll need to load the counts. If you want to build a conditional VAE, you'll also need to load the batch and/or the labels.

In [ ]:
from scvi.data import AnnDataManager
from scvi.data.fields import CategoricalObsField, LayerField
from scvi.dataloaders import AnnDataLoader


# which fields of th AnnData do you need?
anndata_fields = [
    LayerField(registry_key="counts", layer="counts", is_count_data=True),  # raw counts
    CategoricalObsField(
        registry_key="batch", attr_key="batch"
    ),  # if needed: catagorical field in adata.obs for the batch info
    CategoricalObsField(
        registry_key="cell_type", attr_key="cell_type"
    ),  # if needed: catagorical field in adata.obs for the cell type label info
]

adata_manager = AnnDataManager(fields=anndata_fields)
adata_manager.register_fields(adata)

# print out the summary stats of the data: how many batches, cell types, genes..
adata_manager.summary_stats

In [ ]:
n_cells = len(adata)
frac_train = 0.8


# Randomly split the cells into training and testing sets
train_index = np.random.choice(n_cells, size=int(frac_train * n_cells), replace=False)
train_mask = np.empty(shape=(n_cells,), dtype=bool)
train_mask[:] = False
train_mask[train_index] = True
val_mask = np.logical_not(train_mask)

print(f"{train_mask.sum()} training cells")
print(f"{val_mask.sum()} validation cells")

In [ ]:
# initialize an AnnDataLoader which will iterate over our anndata
adl_train = AnnDataLoader(
    adata_manager, shuffle=False, batch_size=BATCH_SIZE, indices=train_index
)
adl_validation = AnnDataLoader(
    adata_manager, shuffle=False, batch_size=BATCH_SIZE, indices=val_mask
)


# get the first batch of data
ex_data_batch = next(iter(adl_train))


print(ex_data_batch["counts"].shape)  # shape is batch_size x n_genes
print(ex_data_batch["batch"].shape)  # shape is batch_size x 1
print(ex_data_batch["cell_type"].shape)  # shape is batch_size x 1

In [4]:
from model import VAE

# count_model =

## 3.2 Model training

<div class="alert alert-block alert-info">
<b>Question 5</b>
Complete the code in `train.py` to train the model. You should use the analytical expressions of the KL divergence and the reconstruction term that you derived in Question 2.
</div>

A common practise is to warmup the KL divergence term in the ELBO. This is done by multiplying the KL term by a weight, which is gradually increased from 0 to 1 over the course of training. This is done to prevent the model from learning a trivial latent representation, as discussed in [this paper](https://arxiv.org/pdf/1602.02282.pdf).

A `_compute_kl_weight` function is provided in `utils.py` to implement this.

In [86]:
from train import train_loop

from torch.optim.lr_scheduler import StepLR, ExponentialLR  # try more!

You are encouraged to try different architectures and hyperparameters. 
Try different optimizers and schedulers - as listed in [torch.optim](https://pytorch.org/docs/stable/optim.html). 

In [5]:
# optimizer =

# scheduler =


# train_kl, train_recon, test_kl, test_recon = train_loop(...)

In [11]:
# scheduler.get_last_lr()

Plot the learning curves

In [6]:
# plt.plot(train_kl, label="Train KL")
# plt.plot(test_kl, label="Val KL")
# plt.legend()
# plt.show()

In [7]:
# plt.plot(train_recon, label="Train Recon")
# plt.plot(test_recon, label="Val Recon")
# plt.legend()
# plt.show()

## 3.3 Get the latent representation, compare with PCA

Once the model is trained, we can use the encoder to get the latent representation of the cells. We can then compare the VAE latent representation with the PCA representation.

Compared to the VAE, the PCA is a linear method that projects the data into a lower-dimensional space by maximizing the variance of the projected data. It is a deterministic method that does not model the data distribution. You may refresh your memory on PCA [here](https://en.wikipedia.org/wiki/Principal_component_analysis).

<div class="alert alert-block alert-info">
<b>Question 6</b>
Implement the `get_latent_representation` function in `model.py` to get the latent representation of the cells.
</div>

Then you will visualize the latent and PCA spaces using UMAP - a dimensionality reduction technique that is often used in single-cell analysis to visualize the data in 2D or 3D.
If you are not familiar with UMAP, you can read more about it [here](https://www.scdiscoveries.com/blog/knowledge/what-is-a-umap-plot/).

In [91]:
from model import get_latent_representation

# adata.obsm["X_vae"] = get_latent_representation(...)

In [8]:
sc.pp.neighbors(adata, use_rep="X_vae")
sc.tl.umap(adata, min_dist=0.3)

sc.pl.umap(
    adata,
    color=["cell_type"],
    frameon=False,
)
sc.pl.umap(
    adata,
    color=["donor", BATCH_KEY],
    ncols=2,
    frameon=False,
)

In [ ]:
# run PCA then generate UMAP plots
sc.tl.pca(adata)
sc.pp.neighbors(adata, n_pcs=30, n_neighbors=20)
sc.tl.umap(adata, min_dist=0.3)

sc.pl.umap(
    adata,
    color=["cell_type"],
    frameon=False,
)
sc.pl.umap(
    adata,
    color=["donor", BATCH_KEY],
    ncols=2,
    frameon=False,
)

## 3.4 Benchmark with scIB




Now we can provide quantitative measurement of the integration performance, using metrics developped in the paper [Benchmarking atlas-level data integration in single-cell genomics](https://www.nature.com/articles/s41592-021-01336-8).

Scalable implementation of these metrics are available in the [scib-metrics](https://scib-metrics.readthedocs.io/en/stable/index.html) package.

In [94]:
from scib_metrics.benchmark import Benchmarker

In [9]:
# compute a set of metrics using scib.


bm = Benchmarker(
    adata,
    batch_key="batch",
    label_key="cell_type",
    embedding_obsm_keys=["X_pca", "X_vae"],
    n_jobs=-1,
)
bm.benchmark()

In [10]:
bm.plot_results_table(min_max_scale=False)

You should aim for a better performance than PCA in both Batch correction and Bio Conservation metrics. Find the set of parameters/hyperparameters that gives you the best performance 😊

## 3.5 Bonus: Implement batch correction in the VAE

The idea here is to concatenate to the decoder imput the batch information of the cells. This can be done by adding a batch [embedding layer](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html) to the decoder. The batch embedding layer is a learnable matrix of size (n_batches, n_latent). Alternatively, you can embedd the batch indices using [one-hot](https://pytorch.org/docs/stable/generated/torch.nn.functional.one_hot.html) vectors. The batch information is then concatenated to the latent representation before feeding it to the decoder. This is a simple way to correct for batch effects in the data. 


<div class="alert alert-block alert-info">
<b>Question 7</b>
Modify the decoder to include batch information. Train the model and compare the latent representations with and without batch correction using the scIB metrics.
</div>